In [18]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd


In [29]:
#url de libros Uptc
url = 'https://editorial.uptc.edu.co/catalogo.html?product_list_limit=60'
bucle = 'https://editorial.uptc.edu.co/catalogo.html?p=2&product_list_limit=60'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap=soup.find('div',class_="sa-product-container-v3 grid")
productos_url = scrap.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

url_libros = [url.find('a')["href"] for url in productos_url]
urls = pd.DataFrame(url_libros,columns=['url'])
for i in range(2,6):
    url = f'https://editorial.uptc.edu.co/catalogo.html?p={i}&product_list_limit=60'
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html')
    productos_url = soup.find('div',class_="sa-product-container-v3 grid").find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
    url_libros = [url.find('a')["href"] for url in productos_url]
    urls = pd.concat([urls,pd.DataFrame(url_libros,columns=['url'])])



In [32]:
# urls.to_csv('urls.csv',index=False)
urls.shape

(298, 1)

In [5]:
url_libros[1]

'https://editorial.uptc.edu.co/gpd-introduccion-al-calculo-de-probabilidades-para-estudiantes-de-ingenieria-9789586609449-680d0a2fc352d.html'

In [10]:
titulos = []
autores = []
palabras_clave = []
precios = []
descripcion = []


res = requests.get(url_libros[1])
soup = BeautifulSoup(res.text,'html')

scrap=soup.find('div',class_="product-info-main")
titulos.append(scrap.find('div',class_="col-xs-12").find('h2').text)
## busquemos lo autores
temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_author' in a['href'] ]
autores.append(temp1)
# busca las palabras clave
temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' in a['href'] ]
palabras_clave.append(temp1)
# DOi
try:
    doi = temp[1].find_all('a')[-1].text
except Exception as e:
    print(f'No se encontro doi: {e}')
    doi = None

# precios
try:
    temp = [a for a in scrap.find_all('div',class_='price-box price-final_price')]
    temp1 = [a.text.strip() for a in temp]
    precios.append(temp1)
except Exception as e:
    print(f'No se encontro precio: {e}')
    precios.append('Gratuito')
# descripcion
descripcion.append(scrap.find('div',class_='description-wrapper').text)
# eBook
try:
    scrap = soup.find('div',class_="product-info-metadata")
    temp1 = [a.text for a in scrap.find_all('strong')]
    temp2 = [a.text for a in scrap.find_all('span')]
    metadata_ebook ={c:v for c,v in zip(temp1,temp2)}
except Exception as e:
    print(f'No se encontro metadata_ebook: {e}')
    metadata_ebook = None
#impreso
try:
    scrap = soup.find_all('div',class_="product-info-metadata")[1]
    temp1 = [a.text for a in scrap.find_all('strong')]
    temp2 = [a.text for a in scrap.find_all('span')]
    metadata_imp ={c:v for c,v in zip(temp1,temp2)}
except Exception as e:
    print(f'No se encontro metadata_imp: {e}')
    metadata_imp = None
datos= {
        "title": titulos,
        "colaboradores":autores,
        "precio":precios,
        "palabras clave":palabras_clave,
        "doi":doi,
        "descripcion":descripcion,
        "metadata_ebook":metadata_ebook,
        "metadata_imp":metadata_imp
    }

No se encontro metadata_imp: list index out of range


In [12]:
datos

{'title': ['Introducción al cálculo de probabilidades para estudiantes de ingeniería'],
 'colaboradores': [['Alex L. Rojas']],
 'precio': [['']],
 'palabras clave': [['Espacio muestral',
   'Probabilidad',
   'Probabilidad condicional',
   'Variables aleatorias',
   'Funciones de distribución',
   'Función de fiabilidad',
   'Función cuantil']],
 'doi': 'Función cuantil',
 'descripcion': ['\nEl propósito de este libro es servir de texto guía en cursos de probabilidad para estudiantes de ingeniería. Por esta razón, los conceptos presentados son principalmente aplicados a la solución de problemas en diversos campos de la ingeniería. En el primer capítulo se presentan los conceptos básicos de matemáticas que facilitarán la comprensión del resto del material presentado. En el segundo capítulo se presentan las definiciones básicas de probabilidad y probabilidad condicional, junto a las principales reglas para hacer cálculos de probabilidades. En el tercer capítulo se introduce el concepto d

In [78]:
scrap=soup.find('div',class_="product-info-main")
temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
temp[1].find_all('a')[-1].text

'10.19053/uptc.9789586609265'

In [45]:
scrap = soup.find_all('div',class_="product-info-metadata")[1]

In [51]:
scrap.find_all('strong')

[<strong>DOI:</strong>,
 <strong>Detalle de formato:</strong>,
 <strong>Estado:</strong>,
 <strong>ISBN-13:</strong>,
 <strong>Tipo de contenido principal:</strong>,
 <strong>Idioma del texto:</strong>,
 <strong>Tamaño (cm):</strong>,
 <strong>Número de páginas del contenido principal:</strong>,
 <strong>Sello editorial: </strong>,
 <strong>Tipo de edición: </strong>,
 <strong>Número de edición: </strong>,
 <strong>Ciudad de publicación:</strong>,
 <strong>País de publicación:</strong>,
 <strong>Fecha de publicación:</strong>,
 <strong>Tipo de restricción de venta:</strong>,
 <strong>SIIEL SKU:</strong>,
 <strong>Precio: </strong>]

In [53]:
temp1 = [a.text for a in scrap.find_all('strong')]
temp2 = [a.text for a in scrap.find_all('span')]
metadata_imp ={c:v for c,v in zip(temp1,temp2)}

In [54]:
metadata_imp

{'DOI:': 'Tapa blanda o Bolsillo',
 'Detalle de formato:': 'Activo',
 'Estado:': '9789586609265',
 'ISBN-13:': 'Texto (legible a simple vista)',
 'Tipo de contenido principal:': 'Español',
 'Idioma del texto:': '14 x 21  x 0.6',
 'Tamaño (cm):': '106 Páginas',
 'Número de páginas del contenido principal:': 'universidad Pedagógica y Tecnológica de Colombia',
 'Sello editorial: ': 'Nueva edición',
 'Tipo de edición: ': '1',
 'Número de edición: ': 'Tunja',
 'Ciudad de publicación:': 'Colombia',
 'País de publicación:': '2025',
 'Fecha de publicación:': 'Exclusivo para un punto o canal de venta',
 'Tipo de restricción de venta:': '419307',
 'SIIEL SKU:': 'COP $40.000'}

In [44]:
metadata_ebook

{'Detalle de formato:': 'Digital (suministrado electrónicamente)',
 'Formato:': 'EPUB',
 'Estado:': 'Activo',
 'ISBN-13:': '9789586609272',
 'Tipo de contenido del producto:': 'Texto (legible a simple vista)',
 'Protección técnica de publicación digital:': 'DRM',
 'Tipo de restricción de uso en publicaciones digitales:': 'Share',
 'Permiso de uso en publicaciones digitales:': 'Permitido, con limitaciones',
 'Idioma del texto:': 'Español',
 'Tamaño del archivo:': '1,593 Kbytes',
 'Sello editorial: ': 'Universidad Pedagógica y Tecnológica de Colombia',
 'Tipo de edición: ': 'Nueva edición',
 'Número de edición: ': '1',
 'Ciudad de publicación:': 'Tunja',
 'País de publicación:': 'Colombia',
 'Fecha de publicación:': '2025',
 'Tipo de restricción de venta:': 'Exclusivo para un punto o canal de venta',
 'Precio: ': 'COP $30.000'}

In [41]:
len(temp1) ,len(temp2)

(18, 18)

In [39]:
for a in scrap.find_all('span'):
    print(a.text)

Digital (suministrado electrónicamente)
EPUB
Activo
9789586609272
Texto (legible a simple vista)
DRM
Share
Permitido, con limitaciones
Español
1,593 Kbytes
Universidad Pedagógica y Tecnológica de Colombia
Nueva edición
1
Tunja
Colombia
2025
Exclusivo para un punto o canal de venta
COP $30.000


In [38]:
temp

['Detalle de formato:',
 'Formato:',
 'Estado:',
 'ISBN-13:',
 'Tipo de contenido del producto:',
 'Protección técnica de publicación digital:',
 'Tipo de restricción de uso en publicaciones digitales:',
 'Permiso de uso en publicaciones digitales:',
 'Idioma del texto:',
 'Tamaño del archivo:',
 'Sello editorial: ',
 'Tipo de edición: ',
 'Número de edición: ',
 'Ciudad de publicación:',
 'País de publicación:',
 'Fecha de publicación:',
 'Tipo de restricción de venta:',
 'Precio: ']

In [115]:
temp1

['Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento']

In [73]:
temp1

['Otto Caro Niño',
 'Orlando Díaz Parra',
 'Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento',
 '10.19053/uptc.9789586609265']

In [9]:
res.status_code

200

 tenemos un estatus 200 lo cual es correcto la solicitud
 

In [4]:
soup = BeautifulSoup(res.text,'html')

In [19]:
soup.find('title').get_text()

'Catálogo - UPTC'

In [22]:
datos=soup.find('div',class_="sa-product-container-v3 grid")


In [26]:
productos = datos.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

In [ ]:
a = {
    titulo: 
    colaboradores:
    ano:
    precio:
}

In [48]:
title =[i.find('span').get_text(strip=True) for i in productos]

In [62]:
ano = [i.find('div',class_='product-list-edition').get_text(strip=True) for i in productos]

In [76]:
productos[].find('span',class_='minimal-price').get_text()

'\nImpreso\n\n\nCOP $40.000\n\n\n'

In [101]:
datos = []
for i in productos:
    a = {
        "title":i.find('span').get_text(strip=True),
        "colaboradores":list(i.find('div',class_='sa-product-author-v3').stripped_strings)[1],
        "precio": i.find('span',class_='minimal-price').get_text().replace("\n",""),
        "ano": i.find('div',class_='product-list-edition').get_text(strip=True)
    }
    datos.append(a)


In [102]:
datos

[{'title': 'Transporte de fluidos bombas centrífugas',
  'colaboradores': 'Otto Caro Niño                                                                                     y otros',
  'precio': 'eBookCOP $30.000',
  'ano': '2025'},
 {'title': 'Introducción al cálculo de probabilidades para estudiantes de ingeniería',
  'colaboradores': 'Alex L. Rojas',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Derechos colectivos en Colombia:',
  'colaboradores': 'Luis Bernardo Díaz Gamboa                                                                                     y otros',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Resistencias y voces de las mujeres:',
  'colaboradores': 'Celina de Jesús Trimiño Velásquez                                                                                     y otros',
  'precio': 'ImpresoCOP $40.000',
  'ano': '2025'},
 {'title': 'Manual de prácticas pedagógicas para la formación de maestros.',
  'colaboradores': 'Myriam Cecilia

## 1.2 Encontrando las url de cada producto.

+ esto nos permitira acceder a toda la informacion del producto 

In [ ]:
productos = soup.find('div',class_='sa-product-container-v3 grid').find_all('li')
urls=[url.find('a') ['href'] for url in productos]

In [18]:
len(productos)

68

In [31]:
productos[0].find('a')

<a href="https://editorial.uptc.edu.co/gpd-transporte-de-fluidos-bombas-centrifugas-9789586609265-67ebeccc7db73.html">
<img alt="Portada de la publicación Transporte de fluidos bombas centrífugas" src="https://simehbucket.s3.amazonaws.com/images/415ebe2115322046c17eea71821248a5-medium.jpg"/>
</a>